# Data Cleaning

---

## Step 1: Import Libraries

In [1]:
import pandas as pd
import re
import numpy as np

## Step 2: Load the Dataset

In [2]:
# Load the dataset
file_path = "StayFree Export - Total Usage - 12_19_24.xls"  # Replace with your actual file path
output_path = "usage_data_cleaned.xlsx"
df = pd.read_excel(file_path,sheet_name="Usage Time")
df.head()

,Unnamed: 0,Device,"December 8, 2024","December 9, 2024","December 10, 2024","December 11, 2024","December 12, 2024","December 13, 2024","December 14, 2024","December 15, 2024","December 16, 2024","December 17, 2024","December 18, 2024","December 19, 2024",Total Usage
0,a.co,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,2s,0s,2s
1,ActionDash,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,1m 47s,44s,2m 32s
2,Airtel,NaN,0s,0s,0s,0s,0s,12s,0s,0s,0s,0s,0s,0s,12s
3,amazon.ca,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,2s,0s,2s
4,analytics.twitter.com,NaN,0s,0s,0s,0s,0s,0s,0s,0s,0s,0s,6s,0s,6s


## Step 3: Remove Dynamic Text Information
We will define a function to remove dynamic creation dates, time, and the "Created by StayFree" text.

In [3]:
# Function to remove dynamic date, time, and "Created by" text
def remove_creation_info(text):
    if isinstance(text, str):
        # Regex pattern to match "Creation date: MM/DD/YY HH:MM:SS"
        text = re.sub(r"Creation date:\s*\d{1,2}/\d{1,2}/\d{2,4}\s*\d{1,2}:\d{2}:\d{2}(:\d{2})?", '', text)
        # Regex pattern to match "Created by “StayFree”."
        text = re.sub(r"Created by “StayFree”\.", '', text)
    return text

# Apply the remove_creation_info function to the entire DataFrame
df = df.applymap(remove_creation_info)
df.columns.values[0] = 'Apps'


C:\Users\Lakshay\AppData\Local\Temp\ipykernel_11368\163616801.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_creation_info)


## Step 4: Clean Up Unwanted Columns
We will remove any unwanted columns such as those with names like 'Unnamed' or irrelevant columns like 'Device'.

In [4]:
# Step 1: Remove unwanted columns (Unnamed or irrelevant columns)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('Device')]

## Step 5: Convert Time to Seconds
We will define a function to convert time from various formats (such as '3h 4m', '29m', '4s') into total seconds.

In [5]:
# Function to convert time to hours
def convert_to_hours(time_str):
    # If time is '0s' or NaN, return 0
    if pd.isna(time_str) or time_str == '0s':
        return 0.0
    
    # Initialize variables for hours, minutes, and seconds
    hours = minutes = seconds = 0
    
    # Regex pattern to match hours, minutes, and seconds in various formats
    match = re.match(r'(\d+)h\s*(\d+)m\s*(\d+)s', time_str)  # Match hours, minutes, and seconds
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        seconds = int(match.group(3))
    else:
        # Match hours and minutes or hours and seconds
        match = re.match(r'(\d+)h\s*(\d+)m', time_str)  # Match hours and minutes
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2))
        else:
            match = re.match(r'(\d+)h\s*(\d+)s', time_str)  # Match hours and seconds
            if match:
                hours = int(match.group(1))
                seconds = int(match.group(2))
            else:
                # Match minutes and seconds
                match = re.match(r'(\d+)m\s*(\d+)s', time_str)
                if match:
                    minutes = int(match.group(1))
                    seconds = int(match.group(2))
                else:
                    # Match only minutes
                    match = re.match(r'(\d+)m', time_str)
                    if match:
                        minutes = int(match.group(1))
                    else:
                        # Match only seconds
                        match = re.match(r'(\d+)s', time_str)
                        if match:
                            seconds = int(match.group(1))
    
    # Convert everything to hours
    total_hours = hours + (minutes / 60) + (seconds / 3600)
    return total_hours


In [6]:
convert_to_hours("29m")

0.48333333333333334

## Step 6: Clean DataFrame
Remove any rows with 'Total Usage', NaN, or blank values.

In [7]:
# Drop 'Total Usage' and rows with NaN or blank values
df_cleaned = df.drop('Total Usage', axis=1).dropna(how='any')
df_cleaned = df_cleaned.loc[~(df_cleaned == '').any(axis=1)]
df_cleaned = df_cleaned[df_cleaned['Apps'] != 'Total Usage']


## Step 7: Convert DataFrame to Long Format
Now, we will reshape the dataframe from wide to long format.

In [8]:
# Convert the DataFrame to long format
long_format = pd.melt(df_cleaned, id_vars=['Apps'], var_name='Date', value_name='Usage')

## Step 8: Convert Date Column
Convert the Date column into a proper datetime format.

In [9]:
# Convert 'Date' column to datetime format
long_format['Date'] = pd.to_datetime(long_format['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

## Step 9: Apply Time Conversion
Apply the convert_to_seconds function to the 'Usage' column.

In [10]:
# Apply the function to convert time to seconds
long_format['Usage (in hours)'] = long_format['Usage'].apply(convert_to_hours)

In [11]:
long_format.head()

,Apps,Date,Usage,Usage (in hours)
0,a.co,2024-12-08,0s,0.0
1,ActionDash,2024-12-08,0s,0.0
2,Airtel,2024-12-08,0s,0.0
3,amazon.ca,2024-12-08,0s,0.0
4,analytics.twitter.com,2024-12-08,0s,0.0


## Step 10: Add Usage Count Column

In [37]:
# Load the Usage Count data from an Excel sheet
usage_count_df = pd.read_excel(file_path, sheet_name="Usage Count")

# Clean the Usage Count DataFrame
usage_count_df = usage_count_df.applymap(remove_creation_info)  # Apply a cleaning function
usage_count_df = usage_count_df.loc[~(usage_count_df == '').any(axis=1)]  # Remove rows with empty values
usage_count_df = usage_count_df.drop('Device', axis=1)  # Drop the 'Device' column
usage_count_df = usage_count_df.dropna()  # Drop any rows with NaN values
usage_count_df.rename(columns={usage_count_df.columns[0]: 'App Name'}, inplace=True)  # Rename the first column to 'App Name'
usage_count_df = usage_count_df[usage_count_df['App Name'] != 'Total Usage']  # Remove rows where 'App Name' is 'Total Usage'

# Reshape Usage Count DataFrame to long format
df1_long = usage_count_df.melt(
    id_vars=["App Name"], 
    var_name="Date", 
    value_name="Usage Count"
)

# Convert the 'Date' column to a proper format
df1_long["Date"] = pd.to_datetime(df1_long["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

# Merge the long-format Usage Count data with the main dataset
merged_df = pd.merge(
    long_format, 
    df1_long, 
    left_on=["Apps", "Date"], 
    right_on=["App Name", "Date"], 
    how="left"
)

# Drop unnecessary columns and fill missing Usage Count values with 0
merged_df = merged_df.drop(columns=["App Name"]).fillna({"Usage Count": 0})

# Display the final merged DataFrame
merged_df

C:\Users\Lakshay\AppData\Local\Temp\ipykernel_11368\802318542.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  usage_count_df = usage_count_df.applymap(remove_creation_info)  # Apply a cleaning function


,Apps,Date,Usage,Usage (in hours),Usage Count
0,a.co,2024-12-08,0s,0.000000,0.0
1,ActionDash,2024-12-08,0s,0.000000,0.0
2,Airtel,2024-12-08,0s,0.000000,0.0
3,amazon.ca,2024-12-08,0s,0.000000,0.0
4,analytics.twitter.com,2024-12-08,0s,0.000000,0.0
...,...,...,...,...,...
847,YouTube,2024-12-19,0s,0.000000,0.0
848,YouTube,2024-12-19,0s,0.000000,1.0
849,YouTube,2024-12-19,1s,0.000278,0.0
850,YouTube,2024-12-19,1s,0.000278,1.0


## aad device unlock columns

In [46]:
unlock_df = pd.read_excel(file_path, sheet_name="Device Unlocks")
unlock_df = unlock_df.applymap(remove_creation_info)
unlock_df = unlock_df.dropna()  # Drop any rows with NaN values
unlock_df.rename(columns={unlock_df.columns[0]: 'Unlock'}, inplace=True)
unlock_df = unlock_df.drop(columns=["Unlock", "Total Usage"])
# Reshape the data to long format
df_long = unlock_df.melt(var_name="Date", value_name="Device Unlock")

# Convert 'Date' column to proper date format
df_long["Date"] = pd.to_datetime(df_long["Date"], format="%B %d, %Y").dt.strftime("%Y-%m-%d")

# Rename columns for clarity
df_long.columns = ["Date", "Device Unlock"]
df_long

C:\Users\Lakshay\AppData\Local\Temp\ipykernel_11368\3850063761.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unlock_df = unlock_df.applymap(remove_creation_info)


,Date,Device Unlock
0,2024-12-08,266.0
1,2024-12-09,507.0
2,2024-12-10,477.0
3,2024-12-11,447.0
4,2024-12-12,453.0
5,2024-12-13,439.0
6,2024-12-14,293.0
7,2024-12-15,353.0
8,2024-12-16,299.0
9,2024-12-17,271.0


In [47]:
merged_df = pd.merge(
    merged_df, 
    df_long, 
    on="Date", 
    how="left"
)

# Fill missing values in the 'Device Unlock' column with 0
merged_df["Device Unlock"] = merged_df["Device Unlock"].fillna(0)
merged_df

,Apps,Date,Usage,Usage (in hours),Usage Count,Device Unlock
0,a.co,2024-12-08,0s,0.000000,0.0,266.0
1,ActionDash,2024-12-08,0s,0.000000,0.0,266.0
2,Airtel,2024-12-08,0s,0.000000,0.0,266.0
3,amazon.ca,2024-12-08,0s,0.000000,0.0,266.0
4,analytics.twitter.com,2024-12-08,0s,0.000000,0.0,266.0
...,...,...,...,...,...,...
847,YouTube,2024-12-19,0s,0.000000,0.0,64.0
848,YouTube,2024-12-19,0s,0.000000,1.0,64.0
849,YouTube,2024-12-19,1s,0.000278,0.0,64.0
850,YouTube,2024-12-19,1s,0.000278,1.0,64.0


## Save the Data
You can also save the cleaned DataFrame to an Excel file for further analysis.

In [38]:
# Optionally, save the cleaned DataFrame to an Excel file
merged_df.to_excel(output_path, index=False)